In [ ]:
install.packages('RSQLite', dependencies = T , repos='http://cran.us.r-project.org')

In [ ]:
install.packages('sqldf', dependencies = T , repos='http://cran.us.r-project.org')

<html><b>Including the required libraries</b></html>

In [ ]:
library(RSQLite)
library(sqldf)

<html><b>Connecting to the database.</b></html>

In [ ]:
db = dbConnect(SQLite(), dbname="database.sqlite")

<html><b>Fetching the ratings of the players.</b></html>

In [ ]:
p1 = dbGetQuery(db, "select player_api_id, player_name, MAX(overall_rating) as rating from Player_Attributes join Player using(player_api_id) group by player_api_id order by rating desc")


<html><b>Fetching the sum of the gk attributes of all the players.</b></html>

In [ ]:
p2 = dbGetQuery(db, "select player_api_id, MAX(gk_diving + gk_handling + gk_kicking + gk_positioning + gk_reflexes) AS Sum_GK from Player_Attributes group by player_api_id order by player_api_id")


<html><b>Fetching the number of appearances of all the players.</b></html>

In [ ]:
p3 = dbGetQuery(db, "select player as player_api_id, sum(mycount) as appearance
from (
select home_player_1 as player, count(*) as mycount
from match
group by home_player_1
union all
    
select home_player_2 as player, count(*) as mycount
from match
group by home_player_2
union all
    
select home_player_3 as player, count(*) as mycount
from match
group by home_player_3
union all
    
select home_player_4 as player, count(*) as mycount
from match
group by home_player_4
union all
    
select home_player_5 as player, count(*) as mycount
from match
group by home_player_5
union all

select home_player_6 as player, count(*) as mycount
from match
group by home_player_6
union all

select home_player_7 as player, count(*) as mycount
from match
group by home_player_7
union all

select home_player_8 as player, count(*) as mycount
from match
group by home_player_8
union all

select home_player_9 as player, count(*) as mycount
from match
group by home_player_9
union all

select home_player_10 as player, count(*) as mycount
from match
group by home_player_10
union all

select home_player_11 as player, count(*) as mycount
from match
group by home_player_11
union all

select away_player_1 as player, count(*) as mycount
from match
group by away_player_1
union all

select away_player_2 as player, count(*) as mycount
from match
group by away_player_2
union all

select away_player_3 as player, count(*) as mycount
from match
group by away_player_3
union all

select away_player_4 as player, count(*) as mycount
from match
group by away_player_4
union all

select away_player_5 as player, count(*) as mycount
from match
group by away_player_5
union all

select away_player_6 as player, count(*) as mycount
from match
group by away_player_6
union all

select away_player_7 as player, count(*) as mycount
from match
group by away_player_7
union all

select away_player_8 as player, count(*) as mycount
from match
group by away_player_8
union all

select away_player_9 as player, count(*) as mycount
from match
group by away_player_9
union all

select away_player_10 as player, count(*) as mycount
from match
group by away_player_10
union all

select away_player_11 as player, count(*) as mycount
from match
group by away_player_11

)myTab join Player on Player.player_api_id = myTab.player 
group by player 
order by appearance desc")


<html><b>Fetching the number of leagues a team has played for.</b></html>

In [ ]:
p4= dbGetQuery(db, "select player_api_id, count(distinct league_id) as [No of Leagues] from Match join Player
                    where player_api_id=home_player_1 or player_api_id=home_player_2 or player_api_id=home_player_3 or
                          player_api_id=home_player_4 or player_api_id=home_player_5 or player_api_id=home_player_6 or
                          player_api_id=home_player_7 or player_api_id=home_player_8 or player_api_id=home_player_9 or
                          player_api_id=home_player_10 or player_api_id=home_player_11 or 
                          player_api_id=away_player_1 or player_api_id=away_player_2 or player_api_id=away_player_3 or
                          player_api_id=away_player_4 or player_api_id=away_player_5 or player_api_id=away_player_6 or
                          player_api_id=away_player_7 or player_api_id=away_player_8 or player_api_id=away_player_9 or
                          player_api_id=away_player_10 or player_api_id=away_player_11 
                    group by player_api_id order by count(distinct league_id) desc")


<html><b>Combining all the required columns fetched from the player table into the finalPlayer dataframe.</b></html>

In [ ]:
finalPlayer = sqldf("select * from p1 join p2 using(player_api_id) join p3 using(player_api_id) join p4 using(player_api_id)")
finalPlayer


<html><b>Fetching the sum of the Build up play attributes for a team.</b></html>

In [ ]:
#teams
t1 = dbGetQuery(db, "select team_api_id, team_long_name, MAX(buildUpPlaySpeed + buildUpPlaySpeedClass + buildUpPlayDribbling + buildUpPlayDribblingClass + buildUpPlayPassing + buildUpPlayPassingClass + buildUpPlayPositioningClass) AS BuildUpPlay from Team_Attributes join Team using(team_api_id) group by team_api_id order by BuildUpPlay desc")


<html><b>Fetching the sum of the chance creation attributes for a team.</b></html>

In [ ]:
t2 = dbGetQuery(db, "select team_api_id, MAX(chanceCreationPassing + chanceCreationPassingClass + chanceCreationCrossing + chanceCreationCrossingClass + chanceCreationShooting + chanceCreationShootingClass + chanceCreationPositioningClass) AS ChanceCreation from Team_Attributes group by team_api_id order by ChanceCreation desc")

<html><b>Fetching the sum of the Defence attributes for a team.</b></html>

In [ ]:
t3 = dbGetQuery(db, "select team_api_id, MAX(defencePressure + defencePressureClass + defenceAggression + defenceAggressionClass + defenceTeamWidth + defenceTeamWidthClass + defenceDefenderLineClass) AS Defence from Team_Attributes group by team_api_id order by Defence desc")

<html><b>Fetching the number of goals for a team.</b></html>

In [ ]:
t4 = dbGetQuery(db, "select team_api_id, sum(home_team_goal) as Goal from Match join Team
                    where team_api_id=home_team_api_id
                    group by team_api_id")


t5 = dbGetQuery(db, "select team_api_id, sum(away_team_goal) as Goal from Match join Team
                    where team_api_id=away_team_api_id
                    group by team_api_id")


t6 = sqldf("select team_api_id, t4.Goal+t5.Goal as [no of goal] from t4 join t5 using(team_api_id) order by [no of goal] desc")


<html><b>Combining all the required columns fetched from the team table into the finalTeam dataframe.</b></html>

In [ ]:
finalTeam = sqldf("select * from t1 join t2 using(team_api_id) join t3 using(team_api_id) join t4 using(team_api_id)")
finalTeam

<html><b>Writing the data into Player.csv file.</b></html>

In [ ]:
write.csv(finalPlayer,"Player.csv",row.names=FALSE)

<html><b>Writing the data into Team.csv file.</b></html>

In [ ]:
write.csv(finalTeam,"Team.csv",row.names=FALSE)